In [5]:
import open3d as o3d
import torch
import numpy as np

# Capturing Object from different views

In [3]:
bunny = o3d.io.read_triangle_mesh("/Users/korth/Projects/multiview-robust-clip/data/shapes/bunny.obj")
bunny.compute_vertex_normals()

TriangleMesh with 2503 points and 4968 triangles.

In [7]:
%%capture
bunny = o3d.io.read_triangle_mesh("/Users/korth/Projects/multiview-robust-clip/data/shapes/bunny.obj")
bunny.compute_vertex_normals()

angle = 5

rot_matrix = np.array([[np.cos(np.radians(angle)), 0, -np.sin(np.radians(angle))],
                             [0, 1, 0],
                             [np.sin(np.radians(angle)), 0, np.cos(np.radians(angle))]])

current_angle = 0  
for i in range(360 // angle):
    bunny.rotate(rot_matrix)
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(bunny)
    vis.update_geometry(bunny)
    vis.capture_screen_image(f'cameraparams_{current_angle:03d}.png', do_render=True)

    current_angle += angle


In [64]:
o3d.visualization.draw_plotly([bunny])

# Evaluating CLIP features

In [6]:
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

device = torch.device('mps')

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32").to(device)

path = "/Users/korth/Projects/multiview-robust-clip/notebooks/cameraparams_355.png"
image = Image.open(path)

inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities